# Setup

## Importing necessary libraries

In [ ]:
import zipfile
import os
import pandas as pd

## Decompressing the "Archive" files.

In [ ]:
# Defining the path to the "Archives" folder.
archive_path = "/content/drive/MyDrive/MLOps/archive.zip"
extract_path ="/content/drive/MyDrive/MLOps/Instant_Cart"

# Extract the archive
with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"Files extracted to {extract_path}")

Files extracted to /content/drive/MyDrive/MLOps/Instant_Cart


## Loading the extracted CSV files

In [ ]:
extract_path ="/content/drive/MyDrive/MLOps/Instant_Cart"

In [ ]:
# Defining the paths to the files
orders_path = os.path.join(extract_path, 'orders.csv')
order_products_prior_path = os.path.join(extract_path, 'order_products__prior.csv')
order_products_train_path = os.path.join(extract_path, 'order_products__train.csv')
aisles_path = os.path.join(extract_path, 'aisles.csv')
departments_path = os.path.join(extract_path, 'departments.csv')
products_path = os.path.join(extract_path, 'products.csv')

# Loading the CSV files into DataFrames
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
order_products_train = pd.read_csv(order_products_train_path)
aisles = pd.read_csv(aisles_path)
departments = pd.read_csv(departments_path)
products = pd.read_csv(products_path)

print("Files successfully load!")

Files successfully load!


## Checking Dataframes

In [ ]:
# Checking DataFrame shape and info
print("Orders:")
print(orders.shape)
print(orders.info())
print("")
print("Order Products Prior:")
print(order_products_prior.shape)
print(order_products_prior.info())
print("")
print("Order Products Train:")
print(order_products_train.shape)
print(order_products_train.info())
print("")
print("Aisles:")
print(aisles.shape)
print(aisles.info())
print("")
print("Departments:")
print(departments.shape)
print(departments.info())
print("")
print("Products:")
print(products.shape)
print(products.info())

Orders:
(3421083, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
None

Order Products Prior:
(32434489, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB
None

Order Products Train:
(1384617, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138

# Data Prep

## Aggregate product orders

In this step, I am combining the dataset: order_products_prior.csv and order_products_train files to compute the total frequency of each product.

In [ ]:
# Combining prior and train datasets
all_order_products = pd.concat([order_products_prior, order_products_train])

# Calculating product frequency
product_frequency = all_order_products.groupby('product_id').size().reset_index(name='order_count')

# Getting the top 10,000 most ordered products
top_10k_products = product_frequency.nlargest(10000, 'order_count')
print("Top 10,000 products identified!")

Top 10,000 products identified!


## Filtering orders by Top 10,000 products

In this step, I am filtering the orders to include only those that contain one or more of the top 10,000 products

In [ ]:
# Filtering orders with top 10k products
filtered_orders = all_order_products[all_order_products['product_id'].isin(top_10k_products['product_id'])]

# Getting the list of relevant order IDs
filtered_order_ids = filtered_orders['order_id'].unique()

# Filtering the orders DataFrame
filtered_orders_df = orders[orders['order_id'].isin(filtered_order_ids)]
print(f"Filtered orders to include only top 10,000 products. Remaining orders: {len(filtered_orders_df)}")

Filtered orders to include only top 10,000 products. Remaining orders: 3321331


In [ ]:
print(filtered_orders_df.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')


## Further limiting orders by Size

In the previous step, the total number of orders were reduced to: 3,321,331. Next, I am further limiting the size by retaining orders with a minimum of (5) items to reduce the dataset size further.

In [ ]:
# Counting the number of items in each order
order_item_count = filtered_orders.groupby('order_id').size().reset_index(name='item_count')

# Setting the minimum item threshold (e.g., X = 5)
X = 5

# Filtering orders with at least X items
large_orders = order_item_count[order_item_count['item_count'] >= X]

# Getting the list of valid order IDs
valid_order_ids = large_orders['order_id']

# Filtering the original dataset to keep only the valid order IDs
filtered_orders_df = filtered_orders[filtered_orders['order_id'].isin(valid_order_ids)]

# Counting unique orders in the final filtered dataset
unique_orders_count = filtered_orders_df['order_id'].nunique()

print(f"Filtered down to orders with at least {X} items.")
print(f"Final number of unique orders: {unique_orders_count}")
print(f"Final number of rows (products): {len(filtered_orders_df)}")

Filtered down to orders with at least 5 items.
Final number of unique orders: 2400986
Final number of rows (products): 28368235


## Further limiting order to "Active Users"

Previously, I limited the orders to include only those that contain only 5 products from the top 10k products. We ended with a total size of: 2,400,986. For further reducing the dataset, I am going to focus on users with consistent purchasing behavior. For example, users with more than "Y" total orders. For accomplishing this step, I am going to count the total number of orders per user and filter users with at least 10 orders.

In [ ]:
print(filtered_orders_df.columns)


Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')


In [ ]:
# Merging user_id into filtered_orders_df
filtered_orders_df = pd.merge(
    filtered_orders_df,
    orders[['order_id', 'user_id']],
    on='order_id',
    how='left'
)

# Confirming the user_id column is now included
print(filtered_orders_df.columns)

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id'], dtype='object')


In [ ]:
# Counting the total number of orders per user
user_order_counts = filtered_orders_df.groupby('user_id').size().reset_index(name='order_count')

# Setting the threshold for active users (e.g., Y = 10 orders)
Y = 10
active_users = user_order_counts[user_order_counts['order_count'] >= Y]
active_user_ids = active_users['user_id']

print(f"Number of active users with at least {Y} orders: {len(active_user_ids)}")

Number of active users with at least 10 orders: 184340


In [ ]:
filtered_orders_active_users = filtered_orders_df[filtered_orders_df['user_id'].isin(active_user_ids)]

print(f"Remaining orders: {len(filtered_orders_active_users)}")
print(f"Unique users: {filtered_orders_active_users['user_id'].nunique()}")

Remaining orders: 28313728
Unique users: 184340


In [ ]:
print(f"Unique orders remaining: {filtered_orders_active_users['order_id'].nunique()}")

Unique orders remaining: 2391917


In [ ]:
print(filtered_orders_active_users.columns)

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id'], dtype='object')


In [ ]:
# Saving filtered_orders_active_users DataFram into csv
filtered_orders_active_users.to_csv('/content/drive/MyDrive/MLOps/Instant_Cart/filtered_orders_active_users.csv', index=False)

## Further filtering by top 10 aisles or departments.

On the previous process, we obtained a total of 2,391,917 orders. This time, I am focusing on the most frequently ordered items within the top 10 departments. I will be identifying the top 10 departments with the highest number of orders for later retaining only the orders and products that belong to these top aisles or departments.

In [ ]:
filtered_orders_active_users = pd.read_csv('/content/drive/MyDrive/MLOps/Instant_Cart/filtered_orders_active_users.csv')

In the next step, I am merging aisles.csv and departments.csv with products.csv to enrich the product information with aisle and department details.

In [ ]:
# Merging products with aisles and departments
products_enriched = pd.merge(
    products,
    aisles,
    on='aisle_id',
    how='left'
)
products_enriched = pd.merge(
    products_enriched,
    departments,
    on='department_id',
    how='left'
)

# Verifying the columns in the enriched product dataset
print(products_enriched.columns)

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle',
       'department'],
      dtype='object')


Below, I joing the enriched products information (products_enriched) with the (filtered_orders_active_users) DataFrame.

In [ ]:
# Merging product details into the filtered orders dataset
filtered_orders_with_details = pd.merge(
    filtered_orders_active_users,
    products_enriched,
    on='product_id',
    how='left'
)

# Verifying the merged dataset
print(filtered_orders_with_details.columns)

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'product_name', 'aisle_id', 'department_id', 'aisle', 'department'],
      dtype='object')


Next, I group the data by aisle and department (separate) and calculate the total number of orders. Then, sorting the results in descending order and retain the top 10.

In [ ]:
# Counting orders by aisle
aisle_order_counts = filtered_orders_with_details.groupby('aisle').size().reset_index(name='order_count')

# Getting the top 10 aisles
top_aisles = aisle_order_counts.nlargest(10, 'order_count')
top_aisle_names = top_aisles['aisle']

print(f"Top 10 aisles: {list(top_aisle_names)}")

Top 10 aisles: ['fresh fruits', 'fresh vegetables', 'packaged vegetables fruits', 'yogurt', 'packaged cheese', 'milk', 'water seltzer sparkling water', 'chips pretzels', 'soy lactosefree', 'bread']


In [ ]:
# Counting orders by department
department_order_counts = filtered_orders_with_details.groupby('department').size().reset_index(name='order_count')

# Getting the top 10 departments
top_departments = department_order_counts.nlargest(10, 'order_count')
top_department_names = top_departments['department']

print(f"Top 10 departments: {list(top_department_names)}")

Top 10 departments: ['produce', 'dairy eggs', 'snacks', 'beverages', 'frozen', 'pantry', 'bakery', 'deli', 'canned goods', 'dry goods pasta']


In [ ]:
# Filtering orders for top aisles
filtered_by_aisles = filtered_orders_with_details[filtered_orders_with_details['aisle'].isin(top_aisle_names)]

print(f"Remaining orders after filtering by top aisles: {len(filtered_by_aisles)}")

Remaining orders after filtering by top aisles: 14096649


In [ ]:
# Filtering orders for top departments
filtered_by_departments = filtered_orders_with_details[filtered_orders_with_details['department'].isin(top_department_names)]

print(f"Remaining orders after filtering by top departments: {len(filtered_by_departments)}")

Remaining orders after filtering by top departments: 25702383


In [ ]:
# Verifying aisles or departments in the filtered dataset
print(filtered_by_aisles['aisle'].value_counts())
print(filtered_by_departments['department'].value_counts())

aisle
fresh fruits                     3485989
fresh vegetables                 3378122
packaged vegetables fruits       1686448
yogurt                           1364214
packaged cheese                   905963
milk                              806036
water seltzer sparkling water     720034
chips pretzels                    635021
soy lactosefree                   587908
bread                             526914
Name: count, dtype: int64
department
produce            9133122
dairy eggs         4988600
snacks             2366457
beverages          2168493
frozen             1888991
pantry             1502823
bakery             1042682
deli                940087
canned goods        928885
dry goods pasta     742243
Name: count, dtype: int64


I decided to filter the data by "Top 10 Departments" for the following reasons:

- It aligns with the project's goal of creatinga resuable ML pipeline, as department-level insights generalize better.
- Provides a broader perspective, covering diverse products and trends.
- Reduces the dataset size efficiently while retaining valuable data for high-level analysis.

Next, I filter the dataset by the top 10 departments: produce, dairy eggs, snacks, etc. Then, I save the filtered dataset into .csv format.

In [ ]:
# Paths to the files
output_path = "/content/drive/MyDrive/MLOps/Instant_Cart/filtered_by_top_departments.csv"  # Desired output path

# Top 10 departments
top_departments = [
    "produce", "dairy eggs", "snacks", "beverages", "frozen",
    "pantry", "bakery", "deli", "canned goods", "dry goods pasta"
]

# Filtering the dataset to include only top departments
filtered_by_departments = filtered_orders_with_details[
    filtered_orders_with_details['department'].isin(top_departments)
]

# Saving the filtered dataset
filtered_by_departments.to_csv(output_path, index=False)

# Summary of the filtered dataset
filtered_summary = {
    "Remaining Rows": len(filtered_by_departments),
    "Unique Orders": filtered_by_departments['order_id'].nunique(),
    "Unique Users": filtered_by_departments['user_id'].nunique(),
    "Unique Departments": filtered_by_departments['department'].nunique()
}

print("Filtered dataset saved successfully.")
print(filtered_summary)

Filtered dataset saved successfully.
{'Remaining Rows': 25702383, 'Unique Orders': 2389985, 'Unique Users': 184304, 'Unique Departments': 10}


## Further filtering rarely reordered products





From the previous filtering process, we obtained a total number of unique orders of: 2,389,985. Now, I will proceed to calculate the reorder rate for each product by grouping the data by product_id and taking the mean of the "reordered" column.

In [ ]:
# Loading the filtered dataset
filtered_by_top_departments_path = "/content/drive/MyDrive/MLOps/Instant_Cart/filtered_by_top_departments.csv"
filtered_by_top_departments = pd.read_csv(filtered_by_top_departments_path)

# Calculating reorder rate for each product
product_reorder_rate = filtered_by_top_departments.groupby('product_id')['reordered'].mean().reset_index()
product_reorder_rate.rename(columns={'reordered': 'reorder_rate'}, inplace=True)

# Setting a threshold for rarely reordered products (e.g., 0.2 or 20%)
threshold = 0.2
frequently_reordered_products = product_reorder_rate[product_reorder_rate['reorder_rate'] > threshold]

print(f"Products with a reorder rate above {threshold}: {len(frequently_reordered_products)}")

Products with a reorder rate above 0.2: 8002


In [ ]:
# Filtering the dataset to include only frequently reordered products
filtered_frequent_reorders = filtered_by_top_departments[
    filtered_by_top_departments['product_id'].isin(frequently_reordered_products['product_id'])
]

print(f"Filtered dataset to include frequently reordered products.")
print(f"Remaining Rows: {len(filtered_frequent_reorders)}")
print(f"Unique Orders: {filtered_frequent_reorders['order_id'].nunique()}")
print(f"Unique Products: {filtered_frequent_reorders['product_id'].nunique()}")

Filtered dataset to include frequently reordered products.
Remaining Rows: 25454061
Unique Orders: 2389931
Unique Products: 8002


In [ ]:
# Saving the filtered dataset
output_path_frequent_reorders = "/content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_reorders.csv"
filtered_frequent_reorders.to_csv(output_path_frequent_reorders, index=False)

print(f"Filtered dataset saved at {output_path_frequent_reorders}.")

Filtered dataset saved at /content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_reorders.csv.


## Filtering for frequent buyer

From the previous result, the dataset was only reduced to: 2,389,931 orders. Next, I am filtering by identifying frequent buyey. For example: retain users who order top products at least N times.

### N = 450.
This first dataset will be balanced between most frequent buyers and non active users.

**Note to consider:**

As we increase "N", the focus is only on the most active users, which may bias the model toward frequent buyes and their behavior.

The model might become less generalize to users with lower purchasing activity.

If the dataset is heavily skewed toward a small group of active users, some product-specific might be lost. However, keeping more frequent buyers might still capture sufficient trends highly reordered products.

A piperline trained on a subset of frequent buyers may perform for similr groups but may not generalize to less frequent users or broader audience.

In [ ]:
# Defining the threshold for frequent buyers (e.g., at least N = 10 orders)
N = 450

# Counting the number of orders per user
user_order_counts = filtered_frequent_reorders.groupby('user_id').size().reset_index(name='order_count')

# Filtering users with at least N orders
frequent_buyers = user_order_counts[user_order_counts['order_count'] >= N]
frequent_buyer_ids = frequent_buyers['user_id']

print(f"Number of frequent buyers with at least {N} orders: {len(frequent_buyer_ids)}")

Number of frequent buyers with at least 450 orders: 11033


In [ ]:
# Filtering orders for frequent buyers
filtered_frequent_buyers = filtered_frequent_reorders[
    filtered_frequent_reorders['user_id'].isin(frequent_buyer_ids)
]

print(f"Filtered dataset to include orders from frequent buyers.")
print(f"Remaining Rows: {len(filtered_frequent_buyers)}")
print(f"Unique Orders: {filtered_frequent_buyers['order_id'].nunique()}")
print(f"Unique Users: {filtered_frequent_buyers['user_id'].nunique()}")

Filtered dataset to include orders from frequent buyers.
Remaining Rows: 7588145
Unique Orders: 562019
Unique Users: 11033


In [ ]:
# Saving the filtered dataset
output_path_frequent_buyers = "/content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_buyers_v1.csv"
filtered_frequent_buyers.to_csv(output_path_frequent_buyers, index=False)

print(f"Filtered dataset saved at {output_path_frequent_buyers}.")

Filtered dataset saved at /content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_buyers_v1.csv.


### N = 850

This dataset will be for the less generalize ML model.

In [ ]:
N = 850

# Counting the number of orders per user
user_order_counts = filtered_frequent_reorders.groupby('user_id').size().reset_index(name='order_count')

# Filtering users with at least N orders
frequent_buyers = user_order_counts[user_order_counts['order_count'] >= N]
frequent_buyer_ids = frequent_buyers['user_id']

print(f"Number of frequent buyers with at least {N} orders: {len(frequent_buyer_ids)}")

Number of frequent buyers with at least 850 orders: 2081


In [ ]:
# Filtering orders for frequent buyers
filtered_frequent_buyers = filtered_frequent_reorders[
    filtered_frequent_reorders['user_id'].isin(frequent_buyer_ids)
]

print(f"Filtered dataset to include orders from frequent buyers.")
print(f"Remaining Rows: {len(filtered_frequent_buyers)}")
print(f"Unique Orders: {filtered_frequent_buyers['order_id'].nunique()}")
print(f"Unique Users: {filtered_frequent_buyers['user_id'].nunique()}")

Filtered dataset to include orders from frequent buyers.
Remaining Rows: 2253205
Unique Orders: 137382
Unique Users: 2081


In [ ]:
# Saving the filtered dataset
output_path_frequent_buyers = "/content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_buyers_v2.csv"
filtered_frequent_buyers.to_csv(output_path_frequent_buyers, index=False)

print(f"Filtered dataset saved at {output_path_frequent_buyers}.")

Filtered dataset saved at /content/drive/MyDrive/MLOps/Instant_Cart/filtered_frequent_buyers_v2.csv.


# EDA

## Setup

### Installing necessary libraries

In [ ]:
! pip install pyathena

In [ ]:
!pip install awswrangler

In [ ]:
!pip install seaborn

In [ ]:
from pyathena import connect

In [ ]:
import awswrangler as wr
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Converting InstantCart CSV dataset into Parquet

In [ ]:
csv_path = "s3://sagemaker-us-east-1-209611057751/data-lake/project/filtered_frequent_buyers_v1.csv"
df = pd.read_csv(csv_path)
# Loading CSV from S3

train_df, remaining_df = train_test_split(df, train_size=0.4, random_state=42)

# Split the remaining data into production and temp datasets (66% production, 33% temp)
production_df, temp_df = train_test_split(remaining_df, train_size=0.666666, random_state=42)

# Split the temp data into test and validation datasets (50% test, 50% validation)
test_df, validation_df = train_test_split(temp_df, train_size=0.5, random_state=42)

In [ ]:
# Defining S3 paths
parquet_output_path = "s3://sagemaker-us-east-1-209611057751/data-lake/project/partitioned/"

# Save each split dataset to Parquet with partitioning by 'department'
wr.s3.to_parquet(
    df=train_df,
    path=parquet_output_path + "train/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=production_df,
    path=parquet_output_path + "production/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=test_df,
    path=parquet_output_path + "test/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=validation_df,
    path=parquet_output_path + "validation/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

In [ ]:
# Create the database if it doesn't exist
database_name = "instacart_db"
try:
    # Create the database in AWS Glue
    wr.catalog.create_database(name=database_name)
    print(f"Database '{database_name}' created successfully!")
except Exception as e:
    print(f"Error creating database: {e}")

In [ ]:
# Register the Parquet tables in AWS Glue
table_name = "instacart_orders"

wr.catalog.create_parquet_table(
    database=database_name,
    table=table_name,
    path=parquet_output_path + "train/",
    columns_types={
        "order_id": "bigint",
        "product_id": "bigint",
        "add_to_cart_order": "int",
        "reordered": "int",
        "user_id": "bigint",
        "product_name": "string",
        "aisle_id": "int",
        "department_id": "int",
        "aisle": "string",
        "department": "string"
    },
    partitions_types={"department": "string"},
    description="Partitioned Instacart orders dataset for optimized Athena queries."
)

print("Partitioned Parquet table registered in AWS Glue successfully.")

### Setting up Database for InstantCart

In [ ]:
from pyathena import connect

# Defineing AWS Resources
bucket_name = "sagemaker-us-east-1-209611057751"
region = "us-east-1"
database_name = "instacart_db"
table_name = "instacart_orders"
s3_data_location = f"s3://{bucket_name}/data-lake/project/partitioned/train/"  # Using partitioned dataset

# Defining Athena Staging Directory
s3_staging_dir = f"s3://{bucket_name}/athena/instacart_staging/"

# Creating Athena Connection
try:
    conn = connect(s3_staging_dir=s3_staging_dir, region_name=region)
    cursor = conn.cursor()
    print("Connected to Athena successfully.")
except Exception as e:
    print("Error connecting to Athena:", e)

# Creating Database
create_db_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"
cursor.execute(create_db_query)
print(f"Database '{database_name}' created successfully!")

# Verifying Database Creation
cursor.execute("SHOW DATABASES")
databases = [row[0] for row in cursor.fetchall()]
if database_name in databases:
    print(f"Database '{database_name}' exists!")

### Creating Athena database

In [ ]:
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    order_id BIGINT,
    product_id BIGINT,
    add_to_cart_order INT,
    reordered INT,
    user_id BIGINT,
    product_name STRING,
    aisle_id INT,
    department_id INT,
    aisle STRING
)
PARTITIONED BY (department STRING)  -- Partitioned by 'department'
STORED AS PARQUET
LOCATION 's3://sagemaker-us-east-1-209611057751/data-lake/project/partitioned/train/'
TBLPROPERTIES ('parquet.compression'='SNAPPY');
"""

# Execute Table Creation Query
cursor.execute(create_table_query)
print(f"Table '{table_name}' created successfully in database '{database_name}'.")

In [ ]:
# Running MSCK REPAIR to Load Partitions
cursor.execute(f"MSCK REPAIR TABLE {database_name}.{table_name}")
print("Partitions updated successfully.")

In [ ]:
cursor.execute("SHOW DATABASES")
databases = [row[0] for row in cursor.fetchall()]
if database_name in databases:
    print(f"Database '{database_name}' exists in Athena!")
else:
    print(f"Database '{database_name}' does not exist.")

In [ ]:
# Running a Sample Query to Verify Data
test_query = f"SELECT count(*) FROM {database_name}.{table_name} ;"
cursor.execute(test_query)
rows = cursor.fetchall()

print("Sample Query Results:")
for row in rows:
    print(row)

In [ ]:
cursor.execute(f"SHOW PARTITIONS {database_name}.{table_name}")
partitions = cursor.fetchall()
if partitions:
    print(f"Partitions found in table '{table_name}': {partitions}")
else:
    print(f"No partitions found in table '{table_name}'.")

## Checking the Total Orders and Unique Users

In [ ]:
# First Query: Total Orders & Unique Users
query = f"""
SELECT
    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(DISTINCT user_id) AS unique_users
FROM {database_name}.{table_name};
"""

# Executing query
cursor.execute(query)
rows = cursor.fetchall()

# Printing results
print("Total Orders & Unique Users:")
for row in rows:
    print(row)

## Top 10 Most Ordered Products

In [ ]:
query = f"""
SELECT product_name, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY product_name
ORDER BY total_orders DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Most Ordered Products:")
for row in rows:
    print(row)

In [ ]:
# ===========================
#  Bar Chart: Top 10 Most Ordered Products
# ===========================

products = ["Banana", "Bag of Organic Bananas", "Organic Strawberries", "Organic Hass Avocado",
            "Organic Baby Spinach", "Organic Raspberries", "Organic Avocado", "Organic Whole Milk",
            "Limes", "Large Lemon"]
total_orders = [52073, 46964, 39311, 31555, 29459, 21479, 19582, 19211, 17136, 16719]

# Creating DataFrame
df_products = pd.DataFrame({"Product Name": products, "Total Orders": total_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Product Name", data=df_products, palette="viridis")
plt.xlabel("Total Orders")
plt.ylabel("Product Name")
plt.title("Top 10 Most Ordered Products")
plt.show()

### Insights

Overall it can be seen that perishable products are the most ordered products and generally follow a trend of the quicker the product spoils the more often it is ordered. This makes sense as the customers most likely are making smaller orders of those products so as to not let them spoil by having them sit on their counters or refrigerators for an extended period of time. A condensed list of some notable findings can be seen in the list below:

*   Bananas are the most ordered item with 52073 orders.
*   Organic produce is extremely popular making up 7 out of 10 most ordered items.
*   Fruits and vegitables make up 9 out of the top 10 most ordered items.
*   All of the top 10 items spoil within a matter of weeks after opening.

## Reorder Rate per Product

In [ ]:
query = f"""
SELECT
    product_name,
    COUNT(*) AS total_orders,
    SUM(reordered) AS total_reorders,
    ROUND(100.0 * SUM(reordered) / COUNT(*), 2) AS reorder_rate
FROM {database_name}.{table_name}
GROUP BY product_name
ORDER BY reorder_rate DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Products with Highest Reorder Rate:")
for row in rows:
    print(row)

### Insights for Top 10 Products with Highest Reorder Rate:

It is notable that all of the most reordered items have a 100% reorder rate, indicating that customers who order those items have a strong preference for them specifically. Additional insights as they relate to each product can be seen below:
*   **Thirst Quencher Caffeine-Free Naturally Flavored Citrus Soda** and **Smoked Whitefish Salad** have the highest reorder frequency, highlighting that these items are consistently chosen by customers who keep coming back for more.
*   **100% Lactose-Free Milk** has a **100% reorder rate**, indicating it is a highly demanded product among lactose-intolerant customers who consistently reorder it.
*   **Seltzer Water** and **Sparkling Water, Bottles** as well as **Premium Lots of Pulp Orange Juice** are all drink product meaning that customers who order these product may have a strong preference when it comes to the types of drinks and the brands that they consume.

## Orders by Department

In [ ]:
query = f"""
SELECT department, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY department
ORDER BY total_orders DESC;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Orders by Department:")
for row in rows:
    print(row)

In [ ]:
# ===========================
# Bar Chart: Orders by Department
# ===========================

# Data from previous query (Department, Total Orders)
departments = ["produce", "dairy eggs", "snacks", "beverages", "frozen",
               "pantry", "bakery", "deli", "canned goods", "dry goods pasta"]
total_orders = [1116850, 619999, 294213, 234613, 201359, 138924, 127884, 114349, 102416, 84651]

# Creating DataFrame
df_orders_by_department = pd.DataFrame({"Department": departments, "Total Orders": total_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Department", data=df_orders_by_department, palette="magma")
plt.xlabel("Total Orders")
plt.ylabel("Department")
plt.title("Total Orders by Department")
plt.show()

### Insights

Unsurprisingly, the departments with the most orders generally follow a trend of items that spoil the fastest while adding in a factor of items that are consumed at the highest frequency. Produce and dairy/eggs are atop the list by a sizable margin, followed by items that are consumed on a mostly daily basis, snacks and beverages. Next, frozen items are consumed regularly, but have a longer shelf life so they do not need to be ordered as often. The rest of the top ten departments are almost all non/less perishable items. The exemption to this is deli, which falls into the previously discussed category of items that are consumed regularly and have a shorter shelf life so they are most likely ordered in smaller batches. Additional insights can be seen below in a listed format:
*   Produce is the most ordered department with ~1.11 million orders. This aligns with the earlier Top Ordered Products (bananas, avocados, berries).
*   Dairy & Eggs ranks second with 619K orders.
*   Snacks are the third most popular category (294K orders).
*   Beverages (~234K orders) likely include popular items like bottled water, juices, and coffee some of which were in the most reordered top 10.
*   Frozen Foods (~201K orders) suggest customers stock up on frozen essentials.

## Most Popular Aisles

In [ ]:
query = f"""
SELECT aisle, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY aisle
ORDER BY total_orders DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Aisles:")
for row in rows:
    print(row)

In [ ]:
# ===========================
# Bar Chart: Most Popular Aisles
# ===========================

# Data from previous query (Aisle, Total Orders)
aisles = ["fresh fruits", "fresh vegetables", "packaged vegetables fruits", "yogurt",
          "packaged cheese", "milk", "water seltzer sparkling water", "chips pretzels",
          "soy lactosefree", "bread"]
aisle_orders = [450026, 404252, 205492, 184903, 112690, 103953, 78019, 74536, 68786, 65469]

# Creating DataFrame
df_aisles = pd.DataFrame({"Aisle": aisles, "Total Orders": aisle_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Aisle", data=df_aisles, palette="coolwarm")
plt.xlabel("Total Orders")
plt.ylabel("Aisle")
plt.title("Top 10 Most Popular Aisles")
plt.show()

### Insights

The most popular aisles by total orders reflects much of the same findings that have been shown thus far in the data analysis: those items that are the most perishable are ordered the most often. Aisles that have fresh fruits and vegetables rank the highest by a noticeable amount, followed by packaged fruits and vegetables. Then are the dairy products and a few items that fall into the category of snack foods. Other insights can be seen below:

*   Fresh Produce Dominance: Fresh Fruits (450K orders) and Fresh Vegetables (404K orders) are the top two aisles.
*   Combined, these two alone account for over 950K orders, which reinforces that Produce is the top department.
*   Dairy is Highly Popular: Yogurt (184K orders) and Packaged Cheese (112K orders) show strong demand with the addition of Milk (103K orders) confirms that dairy products are household essentials and ordered regularly.
* Beverages have high order numbers: Water, Seltzer, and Sparkling Water (78K orders) ranks #7, showing strong demand for bottled drinks.
*   Snacks & Bread are Key Pantry Items: Chips & Pretzels (74K orders) are among the most frequently purchased snacks. While Bread (65K orders) is typically a staple food.
*   Plant-Based Alternatives: Soy & Lactose-Free Products (68K orders) indicate consumers with dietary restrictions order items that comply with their diet often.

## Reorder Ratio by Department

In [ ]:
query = f"""
SELECT
    department,
    COUNT(*) AS total_orders,
    SUM(reordered) AS total_reorders,
    ROUND(100.0 * SUM(reordered) / COUNT(*), 2) AS reorder_ratio
FROM {database_name}.{table_name}
GROUP BY department
ORDER BY reorder_ratio DESC;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Reorder Ratio by Department:")
for row in rows:
    print(row)

### Insights

Much of the information from the reorder rate by department reflects the trends in the other categories explored so far. The small exception is that beverages and dairy/eggs rank higher on the list than produce. This may be caused by specific preferences in the type of dairy products and beverages as opposed to produce, which may have a higher variance order to order from the same customer. The other departments show bread, deli and snack products are reordered often which aligns with the information that has been gathered with regards to the trend of items that are consumed frequently. Finally, items that have a longer shelf life are reordered less often, possibly indicating that there may be more bulk ordering of these items and there could be orders where these items are not ordered or skipped. Other specifics about the reorder ratio by department can be seen below:

*   Dairy & Eggs Have the Highest Reorder Rate (82.73%)
*   Beverages Rank #2 in Reorders (80.85%)
*   Produce Has a High Reorder Rate (80.71%)
*   Bakery (80.43%) & Deli (78.53%) Show Strong Reorder Loyalty
*   Snacks & Frozen Foods Have Moderate Reorder Rates (~70%)
*   Canned Goods & Dry Goods Have Lower Reorder Rates (~65%)
*   Pantry Has the Lowest Reorder Rate (57.26%)

In [ ]:
# ===========================
# Pie Chart: Reorder Ratio by Department
# ===========================

# Data from previous query (Department, Reorder Ratio)
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
reorder_ratio = [82.73, 80.85, 80.71, 80.43, 78.53, 74.01, 71.87, 65.51, 65.33, 57.26]

# Create DataFrame
df_departments = pd.DataFrame({"Department": departments, "Reorder Ratio": reorder_ratio})

# Plot Pie Chart
plt.figure(figsize=(10, 6))
plt.pie(df_departments["Reorder Ratio"], labels=df_departments["Department"],
        autopct="%1.1f%%", colors=sns.color_palette("viridis", len(departments)), startangle=140)
plt.title("Reorder Ratio by Department")
plt.axis("equal")  # Equal aspect ratio ensures the pie is drawn as a circle
plt.show()

In [ ]:
# ===========================
# Stacked Bar Chart: Reordered vs Non-Reordered Orders by Department
# ===========================

# Data from previous query (Department, Reordered Orders)
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
total_orders = [619999, 234613, 1116850, 127884, 114349, 294213, 201359, 102416, 84651, 138924]
reordered = [512956, 189692, 901358, 102853, 89802, 217758, 144719, 67094, 55299, 79541]
non_reordered = [total - reorder for total, reorder in zip(total_orders, reordered)]

# Create DataFrame
df_reorders = pd.DataFrame({"Department": departments, "Reordered": reordered, "Non-Reordered": non_reordered})

# Plot Stacked Bar Chart (Reordered vs Non-Reordered)
df_reorders.set_index("Department")[["Reordered", "Non-Reordered"]].plot(kind="bar", stacked=True, figsize=(12, 6), colormap="viridis")
plt.xlabel("Department")
plt.ylabel("Number of Orders")
plt.title("Reordered vs Non-Reordered Orders by Department")
plt.legend(["Reordered", "Non-Reordered"])
plt.xticks(rotation=45)
plt.show()

### Analysis for reordered vs non-reordered orders by department:

Looking at the items reordered or not by department shows the same trends that have been seen throughout the entire expository data analysis. Produce and dairy/eggs have the highest reorder rate with the snacks, beverages and frozen goods departments all ranking similarly to each other. Additionally pantry, canned goods and dry pasta goods reinforce the trend of a reordering of frequently consumed non-perishable goods.

## Correlation Analysis

In [ ]:
# Data: Numerical Features for Correlation Analysis
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
total_orders = [619999, 234613, 1116850, 127884, 114349, 294213, 201359, 102416, 84651, 138924]
reordered = [512956, 189692, 901358, 102853, 89802, 217758, 144719, 67094, 55299, 79541]
reorder_ratio = [82.73, 80.85, 80.71, 80.43, 78.53, 74.01, 71.87, 65.51, 65.33, 57.26]

# Creating DataFrame
df_correlation = pd.DataFrame({
    "Total Orders": total_orders,
    "Reordered Orders": reordered,
    "Reorder Ratio (%)": reorder_ratio
})

# Computing Correlation Matrix
correlation_matrix = df_correlation.corr()

# Plotting Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Order Features")
plt.show()

### The correlation matix need to be redone with departments added

## EDA summary

1. Reorder Ratio: strongly correlated with reorders.
2. Total Orders: high correlation with reorder likelihood.
3. User Reorder percerntage: Helps preduct if a user is likely to reorder.
4. Product Popularity: popular items have higher reorders.
5. Department and Aisle reorder ratios: certain categories drive higher reorders

# Feature Engineering

For creating ML Feature features, the following labels will be made:
*   User ID: This tracks individual purchase behavior.
*   Product ID: Helps identify frequently reordered products.
*   Department ID: Some departments have higher reorder rates.
*   Aisle ID: Aisle-level trends impact reorder likelihood.
*   Total Orders: Highly correlated with reorder behavior.
*   Reorder Ratio: Strong predictor of repeat purchases.
*   Total items in Orders: Determines if larger orders influence reorders.
*   User Order Frequency: Identifies frequent vs. occasional buyers.
*   User Reorder Percentage: Determines likelihood of repeat purchases.
*   Product popularity: Captures demand for the product.
*   Department reorder ratio: Some departments have stronger reorder trends.
*   Aisle Reorder ratio: Aisle-specific reorder behavior.
*   Product Reorder trend: Helps detect seasonal or trending products.